#Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment

__author__: Puqi Song

__email__: ps3387@columbia.edu

This file implemented an algorithm which can correct the unfairness for certain groups in machine learning algorithms' predictions. We use the ProPublica COMPAS dataset to test the algorithm. First use Logistic Regression to create training model;  then apply the algorithm into the model and creat new models, calculate both models' accuracy, calibration, false positive rate and false negative rate to see whether the algorithm can increase the model's fairness.

In [1]:
!pip3 install --upgrade pip
!pip install setuptools==57.5.0
!pip3 install dccp
!pip install --upgrade cvxpy
import numpy as np
import pandas as pd
import dccp
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import cvxpy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
print(cp.installed_solvers())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.5.0
    Uninstalling setuptools-57.5.0:
      Successfully uninstalled setuptools-57.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.3.2
    Uninstalling cvxpy-1.3.2:
      Successfully uninstalled cvxpy-1.3.2
['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


The original dataset with the columns we want ('race' only have 'African-American' and 'Caucasian')

In [2]:
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
origin_df = pd.read_csv(url)
df = origin_df.loc[:,["age_cat","sex","race","priors_count","c_charge_degree","two_year_recid"]].query('race in ["African-American","Caucasian"]')
df.reset_index(drop=True,inplace=True)

In [3]:
df

,age_cat,sex,race,priors_count,c_charge_degree,two_year_recid
0,25 - 45,Male,African-American,0,F,1
1,Less than 25,Male,African-American,4,F,1
2,Less than 25,Male,African-American,1,F,0
3,25 - 45,Male,Caucasian,14,F,1
4,25 - 45,Female,Caucasian,0,M,0
...,...,...,...,...,...,...
6145,25 - 45,Male,African-American,0,M,1
6146,Less than 25,Male,African-American,0,F,0
6147,Less than 25,Male,African-American,0,F,0
6148,Less than 25,Male,African-American,0,F,0


Data preprocessing

In [4]:
arr = np.zeros((6150,5))
new_data = pd.DataFrame(arr,columns = ["race", "sex", "priors_count", "c_charge_degree", "two_year_recid"])
features = ["race", "sex", "c_charge_degree"]

lb = preprocessing.LabelBinarizer()
for i in features:
  lb.fit(df[i])
  new_data[i]=lb.transform(df[i])  # Classify the strings into 0 and 1 for "race", "sex", "c_charge_degree"

new_data['priors_count']=df['priors_count']
new_data['two_year_recid']=df['two_year_recid'] # 'priors_count' and 'two_year_recid' do not need to change

age_list=df['age_cat'].value_counts().index.to_list()
for section_name in age_list:
  new_data[f'age_{section_name}']=df['age_cat'].apply(lambda x: int(x ==section_name)) # Assign values by age group and write three new columns

new_data['two_year_recid'][new_data['two_year_recid']==0] = -1 # change 'two_year_recid' from{0,1} to{-1,1}
new_data

,race,sex,priors_count,c_charge_degree,two_year_recid,age_25 - 45,age_Greater than 45,age_Less than 25
0,0,1,0,0,1,1,0,0
1,0,1,4,0,1,0,0,1
2,0,1,1,0,-1,0,0,1
3,1,1,14,0,1,1,0,0
4,1,0,0,1,-1,1,0,0
...,...,...,...,...,...,...,...,...
6145,0,1,0,1,1,1,0,0
6146,0,1,0,0,-1,0,0,1
6147,0,1,0,0,-1,0,0,1
6148,0,1,0,0,-1,0,0,1


In [5]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6150 entries, 0 to 6149
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   race                 6150 non-null   int64
 1   sex                  6150 non-null   int64
 2   priors_count         6150 non-null   int64
 3   c_charge_degree      6150 non-null   int64
 4   two_year_recid       6150 non-null   int64
 5   age_25 - 45          6150 non-null   int64
 6   age_Greater than 45  6150 non-null   int64
 7   age_Less than 25     6150 non-null   int64
dtypes: int64(8)
memory usage: 384.5 KB


Extract feature vectors X ∈ R and class labels y ∈ {−1, 1}.

In [6]:
Y=new_data['two_year_recid']
X=new_data.drop('two_year_recid', axis=1)

Split data into training data and testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
X_train_array = X_train.values
X_test_array = X_test.values
y_train_array = y_train.values
y_test_array = y_test.values
X_train_array_base = X_train_array

Train original model with Logistic Regression

In [ ]:
def train(X_train_array, y_train_array, X_test_array, y_test_array):
  model = LogisticRegression()
  model.fit(X_train_array, y_train_array)

  y_pred = model.predict(X_test_array)
  acc=accuracy_score(y_test_array, y_pred) # accuracy

  raw_scores=model.decision_function(X_test_array)
  probabilities = 1 / (1 + np.exp(-raw_scores))
  prob_true, prob_pred = calibration_curve(y_test_array, probabilities, n_bins=10)
  calibration_error = np.mean(np.abs(prob_pred - prob_true)) # Calibration

  return y_pred, acc, model, calibration_error

y_pred, acc, model, cal_error = train(X_train_array_base, y_train_array, X_test_array, y_test_array)

Define a dataframe to displace y and predicted y with different sensitive vector ('race')

In [ ]:
def score_df(X_test, y_test, y_pred):
  race_test_score=pd.DataFrame(columns=['race','y_real','y_predict'])
  race_test_score['race']=X_test['race']
  race_test_score['y_real']=y_test
  race_test_score['y_predict']=y_pred
  return race_test_score

In [ ]:
df_score = score_df(X_test, y_test, y_pred)
df_score

,race,y_real,y_predict
5630,0,-1,1
644,0,-1,-1
2420,0,1,-1
2753,0,-1,1
714,1,1,-1
...,...,...,...
3970,1,-1,-1
5263,1,1,-1
863,1,-1,-1
104,0,1,1


Define model to calculate DFPR and DFNR

In [ ]:
def cal_pr_nr(df):
  race_African_American_pr=df.loc[(df['race']==0) & (df['y_real']==-1)]
  race_Caucasian_pr=df.loc[(df['race']==1) & (df['y_real']==-1)]
  race_African_American_nr=df.loc[(df['race']==0) & (df['y_real']==1)]
  race_Caucasian_nr=df.loc[(df['race']==1) & (df['y_real']==1)]

  dfpr_base=sum(race_African_American_pr['y_real']!=race_African_American_pr['y_predict'])/race_African_American_pr.shape[0]-sum(race_Caucasian_pr['y_real']!=race_Caucasian_pr['y_predict'])/race_Caucasian_pr.shape[0]
  dfnr_base=sum(race_African_American_nr['y_real']!=race_African_American_nr['y_predict'])/race_African_American_nr.shape[0]-sum(race_Caucasian_nr['y_real']!=race_Caucasian_nr['y_predict'])/race_Caucasian_nr.shape[0]
  return dfpr_base, dfnr_base

In [ ]:
cal = cal_pr_nr(df_score)

Add the 'intercept' column to X

In [ ]:
X_train.insert(0,'intercept',1)
X_train_array = X_train.values

Calculate N, N0, N1

In [ ]:
N = X_train.shape[0]
N0 = X_train[X_train['race']==0].shape[0]
N1 = X_train[X_train['race']==1].shape[0]

Change dataframe into numpy array

In [ ]:
X_train_n0_array = X_train[X_train['race']==0].values
y_train_n0_array = y_train[X_train['race']==0].values
X_train_n1_array = X_train[X_train['race']==1].values
y_train_n1_array = y_train[X_train['race']==1].values

Define dθ(x) and gθ(y, x) consider false positive rates

In [ ]:
def distance(theta, X):
  return X*theta

def g_y_x(X, y, theta):
  dis = distance(theta, X)
  return cp.minimum(0,cp.multiply((1-y)/2, cp.multiply(y,dis)))

Define constraints consider false positive rates and output new parameters to the original model

In [ ]:
def fairness_constraints_1(X, y, X_n0, y_n0, X_n1, y_n1):
  c=0

  num_points, num_features = X.shape
  theta_var = cp.Variable(num_features) #parameter to be updated

  loss = cp.sum(cp.logistic(cp.multiply(-y, X*theta_var)))/ num_points # L(θ)

  cons1 = cp.sum(g_y_x(X_n0, y_n0, theta_var))*((N1/N)/sum(y_n0 == -1))
  cons2 = cp.sum(g_y_x(X_n1, y_n1, theta_var))*((N0/N)/sum(y_n1 == -1))

  thresh = 0
  constraints1 = (cons1 <= cons2 + thresh)
  constraints2 = (cons1 >= cons2 - thresh)

  constraints=[constraints1, constraints2] # Constraints

  prob = cp.Problem(cp.Minimize(loss), constraints)

  prob.solve(method='dccp',solver=cp.ECOS, verbose=False) # Solve problem

  theta_var = np.array(theta_var.value) # New parameter

  return theta_var

New parameter

In [ ]:
theta_new_1=fairness_constraints_1(X_train_array, y_train_array, X_train_n0_array,
                     y_train_n0_array, X_train_n1_array ,y_train_n1_array)

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 46 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 47 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.1

Update the parameter of the old model

In [ ]:
def new_model(model, theta_new):
  model.coef_=theta_new.reshape(-1,1)[1:8].T
  model.intercept_ = theta_new.reshape(-1,1)[0]
  return model

use new model to predict test dataset

In [ ]:
def train_new(model, X_test_array, y_test_array):
  y_pred_new=model.predict(X_test_array)
  acc = accuracy_score(y_test_array, y_pred_new) # Accuracy

  raw_scores=model.decision_function(X_test_array)
  probabilities = 1 / (1 + np.exp(-raw_scores))
  prob_true, prob_pred = calibration_curve(y_test_array, probabilities, n_bins=10)
  calibration_error = np.mean(np.abs(prob_pred - prob_true)) # Calibration


  return y_pred_new, acc, calibration_error


New model

In [ ]:
model_new_1 = new_model(model, theta_new_1)
y_pred_new_1, acc_1 , cal_error_1 = train_new(model_new_1, X_test_array, y_test_array) # new prediction and new accuracy
df_1 = score_df(X_test, y_test, y_pred_new_1)
cal_1 = cal_pr_nr(df_1) # new dfpr and dfnr

Define dθ(x) and gθ(y, x) consider false negative rates

In [ ]:
def g_y_x_2(X, y, theta):
  dis = distance(theta, X)
  return cp.minimum(0,cp.multiply((1+y)/2, cp.multiply(y,dis)))

Define constraints consider false negative rates and output new parameters to the original model

In [ ]:
def fairness_constraints_2(X, y, X_n0, y_n0, X_n1, y_n1):
  c=0

  num_points, num_features = X.shape
  theta_var = cp.Variable(num_features) #parameter to be updated

  loss = cp.sum(cp.logistic(cp.multiply(-y, X*theta_var)))/ num_points # L(θ)

  cons1 = cp.sum(g_y_x_2(X_n0, y_n0, theta_var))*((N1/N)/sum(y_n0 == 1))
  cons2 = cp.sum(g_y_x_2(X_n1, y_n1, theta_var))*((N0/N)/sum(y_n1 == 1))

  thresh = 0
  constraints1 = (cons1 <= cons2 + thresh)
  constraints2 = (cons1 >= cons2 - thresh)

  constraints=[constraints1, constraints2] # Constraints

  prob = cp.Problem(cp.Minimize(loss), constraints)

  prob.solve(method='dccp', solver=cp.ECOS, verbose=False) # Solve problem

  theta_var = np.array(theta_var.value) # New parameter

  return theta_var

New parameter

In [ ]:
theta_new_2=fairness_constraints_2(X_train_array, y_train_array, X_train_n0_array,
                     y_train_n0_array, X_train_n1_array ,y_train_n1_array)

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 49 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 50 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.1

New model

In [ ]:
model_new_2 = new_model(model, theta_new_2)
y_pred_new_2, acc_2, cal_error_2 = train_new(model_new_2, X_test_array, y_test_array) # new prediction and new accuracy
df_2 = score_df(X_test, y_test, y_pred_new_2)
cal_2 = cal_pr_nr(df_2) # new dfpr and dfnr


Define constraints consider both false negative and positive rates and output new parameters to the original model

In [ ]:
def fairness_constraints_3(X, y, X_n0, y_n0, X_n1, y_n1):
  c=0

  num_points, num_features = X.shape
  theta_var = cp.Variable(num_features) # Parameter to be update

  loss = cp.sum(cp.logistic(cp.multiply(-y, X*theta_var)))/ num_points # L(θ)

  cons1 = cp.sum(g_y_x(X_n0, y_n0, theta_var))*((N1/N)/sum(y_n0 == -1))
  cons2 = cp.sum(g_y_x(X_n1, y_n1, theta_var))*((N0/N)/sum(y_n1 == -1))
  cons3 = cp.sum(g_y_x_2(X_n0, y_n0, theta_var))*((N1/N)/sum(y_n0 == 1))
  cons4 = cp.sum(g_y_x_2(X_n1, y_n1, theta_var))*((N0/N)/sum(y_n1 == 1))

  thresh = 0
  constraints1 = (cons1 <= cons2 + thresh)
  constraints2 = (cons1 >= cons2 - thresh)
  constraints3 = (cons3 <= cons4 + thresh)
  constraints4 = (cons3 >= cons4 - thresh)

  constraints=[constraints1, constraints2, constraints3, constraints4] # Constraints

  prob = cp.Problem(cp.Minimize(loss), constraints)

  prob.solve(method='dccp', solver=cp.ECOS, verbose=False) # Solve problem

  theta_var = np.array(theta_var.value) # New parameter

  return theta_var

New parameter

In [ ]:
theta_new_3=fairness_constraints_3(X_train_array, y_train_array, X_train_n0_array,
                     y_train_n0_array, X_train_n1_array ,y_train_n1_array)

/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 52 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:621: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 53 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.1

New model

In [ ]:
model_new_3 = new_model(model, theta_new_3) # new prediction and new accuracy
y_pred_new_3, acc_3, cal_error_3 = train_new(model_new_3, X_test_array, y_test_array)
df_3 = score_df(X_test, y_test, y_pred_new_3)
cal_3 = cal_pr_nr(df_3) # new dfpr and dfnr

Conclusion

In [ ]:
conclusion=pd.DataFrame(columns=['Accuracy', 'DFPR', 'DFNR', 'Calibration'], index=['Original model', 'FPR constraints',
                                'FNR constraints', 'Both constraints'])

conclusion['Accuracy']=[acc, acc_1, acc_2, acc_3]
conclusion['DFPR']=[cal[0], cal_1[0], cal_2[0], cal_3[0]]
conclusion['DFNR']=[cal[1], cal_1[1], cal_2[1], cal_3[1]]
conclusion['Calibration']=[cal_error, cal_error_1, cal_error_2, cal_error_3]

In [ ]:
conclusion

,Accuracy,DFPR,DFNR,Calibration
Original model,0.659298,0.208895,-0.352083,0.034681
FPR constraints,0.650845,0.078065,-0.135417,0.029696
FNR constraints,0.635241,-0.125779,0.116667,0.035362
Both constraints,0.637191,-0.020384,0.050000,0.064850
